In [23]:
import numpy as np
from tensorflow.keras.models import load_model
import lightgbm as lgb
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix


In [183]:
import os
import pandas as pd

# Load clinical data
clinical_data = pd.read_csv('clinical_best.csv')

# Specify the directory containing the image folders
image_directory = 'PREPROCESS_IMAGES/alzheimer_new_2/Axial'

# Initialize lists to store PTIDs, class labels, and corresponding image file paths
ptids = []
class_labels = []
image_files = []

# Traverse through the image directory
for class_label in os.listdir(image_directory):
    class_label_path = os.path.join(image_directory, class_label)
    if os.path.isdir(class_label_path):
        # Traverse through the class label directory
        for file_name in os.listdir(class_label_path):
            # Extract PTID from file name
            ptid = file_name[2:12] 
            #print(ptid)# Extract PTID from index 3 to 12
            ptids.append(ptid)
            class_labels.append(class_label)
            # Construct the full path to the image file
            image_file = os.path.join(class_label_path, file_name)
            image_files.append(image_file)

# Create a DataFrame for image data with PTID, class label, and image file path
image_data = pd.DataFrame({'PTID': ptids, 'Class_Label': class_labels, 'Image_File': image_files})

# Merge clinical data with image data based on PTID
merged_data = pd.merge(clinical_data, image_data, on='PTID', how='inner')
merged_data['Group'] = merged_data.groupby('Class_Label')['Group'].transform('first')


# Display the merged DataFrame
print(merged_data)


      Unnamed: 0  index        PTID   RID  Phase  Group  PTGENDER_-4.0  \
0              1      1  002_S_0413   413  ADNI1    0.0          False   
1              1      1  002_S_0413   413  ADNI1    0.0          False   
2              1      1  002_S_0413   413  ADNI1    0.0          False   
3              1      1  002_S_0413   413  ADNI1    0.0          False   
4              1      1  002_S_0413   413  ADNI1    0.0          False   
...          ...    ...         ...   ...    ...    ...            ...   
3227        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3228        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3229        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3230        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3231        1953   1953  136_S_1227  1227  ADNI1    1.0          False   

      PTGENDER_1.0  PTGENDER_2.0  PTWORK_-4.0  ...  PHC_MEM    PTDOB  PHC_LAN  \
0            False          Tr

In [184]:
class_counts=merged_data["Class_Label"].value_counts()
print(class_counts)

Class_Label
AD    1124
CI    1074
CN    1034
Name: count, dtype: int64


IGNORE--NOT BEING USED

Found 5154 files belonging to 3 classes.
Images shape: (5154, 128, 128, 1)
Labels shape: (5154,)


In [8]:
clinical_data=pd.read_csv('augmented_clinical_data.csv').values

In [27]:
root_folder = "PREPROCESS_IMAGES/alzheimer_new_2/Axial"  # Specify the root folder containing image folders
image_data, labels1 = load_images_with_labels(root_folder)
labels = clinical_data[:,3].astype(int)
labels

Found 5154 files belonging to 3 classes.


array([1, 0, 1, ..., 1, 1, 0])

Continue-- DONE SINCE MODEL CHANGED

In [159]:
cnn_model = load_model('clf_model_final.h5')


CONTINUE

In [104]:
import cv2
# Load trained CNN models
cnn_model = load_model('clf_model_final.h5')

# Load trained LGBM model
lgbm_model = lgb.Booster(model_file='lgbm_model.txt')


preprocessed_images = []

# Loop through each row in the merged DataFrame
for index, row in merged_data.iterrows():
    # Load image
    image = cv2.imread(row['Image_File'], cv2.IMREAD_GRAYSCALE)
    
    # Resize image to (128, 128)
    image_resized = cv2.resize(image, (128, 128))
    
    # Convert grayscale to RGB
    image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
    
    # Normalize pixel values to [0, 1]
    image_normalized = image_rgb / 255.0
    
    # Append the preprocessed image to the list
    preprocessed_images.append(image_normalized)

# Convert the list of preprocessed images to a numpy array
preprocessed_images = np.array(preprocessed_images)

print(preprocessed_images.shape)  # Output the shape of the preprocessed image array


clinical_data = merged_data.drop(columns=['PTID', 'Image_File','PTDOB','Phase','VISDATE','Class_Label','Group'])


# Make predictions using each CNN model
cnn_pred = cnn_model.predict(preprocessed_images)
# Make predictions using more CNN models as needed

# Make predictions using the LGBM model
lgbm_preds = lgbm_model.predict(clinical_data)


[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.

In [166]:
cnn_pred = cnn_model.predict(preprocessed_images)


101/101 ━━━━━━━━━━━━━━━━━━━━ 16s 148ms/step


In [167]:
preprocessed_images.shape

(3232, 128, 128, 3)

Just executing this once, else would have been done above

In [119]:
cnn_pred_labels = np.array([np.argmax(y_) for y_ in cnn_pred])
cnn_pred_labels


array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [80]:
clinical_data.shape


(3232, 118)

iGNORE

In [90]:
class_label_mapping = {'AD': 2, 'MCI': 1, 'CN': 0}
merged_data['Class_Label'] = merged_data['Class_Label'].map(class_label_mapping)

In [145]:
clinical_data = merged_data.drop(columns=['PTID', 'Image_File','PTDOB','Phase','VISDATE','Class_Label','Group'])
#image_data_preprocessed = np.array(merged_data['Image_File'])

#image_data = image_data_preprocessed.reshape(-1, 128, 128, 1)

# Make predictions using each CNN model
cnn_pred = cnn_model.predict(preprocessed_images)
# Make predictions using more CNN models as needed

# Make predictions using the LGBM model
lgbm_preds = lgbm_model.predict(clinical_data)


101/101 ━━━━━━━━━━━━━━━━━━━━ 17s 163ms/step


In [77]:
lgbm_preds

array([[0.65603643, 0.25783164, 0.08613193],
       [0.65603643, 0.25783164, 0.08613193],
       [0.65603643, 0.25783164, 0.08613193],
       ...,
       [0.55846417, 0.34794213, 0.0935937 ],
       [0.55846417, 0.34794213, 0.0935937 ],
       [0.55846417, 0.34794213, 0.0935937 ]])

In [168]:
cnn_pred

array([[9.9897671e-01, 5.6596845e-04, 4.5732732e-04],
       [9.9995899e-01, 4.9829773e-06, 3.5999510e-05],
       [9.9985683e-01, 5.8040455e-06, 1.3728770e-04],
       ...,
       [7.6184165e-06, 9.9999106e-01, 1.2837530e-06],
       [2.1637859e-05, 9.9997139e-01, 6.9874350e-06],
       [2.8938684e-05, 9.9974960e-01, 2.2143488e-04]], dtype=float32)

ALTER AS NEEDED

In [171]:
cnn_weight = 0.8 
lgbm_weight = 0.2

In [84]:
cnn_pred

array([[2.4874930e-07, 3.4144695e-03, 9.9658525e-01],
       [2.1460194e-09, 7.4470742e-04, 9.9925524e-01],
       [2.9079036e-08, 1.0845017e-03, 9.9891555e-01],
       ...,
       [1.2440818e-09, 9.9999940e-01, 6.3088237e-07],
       [3.4537535e-08, 9.9999619e-01, 3.7757097e-06],
       [3.4558404e-08, 9.9999952e-01, 4.1272125e-07]], dtype=float32)

In [169]:
cnn_pred=tf.nn.softmax(cnn_pred)

In [170]:
cnn_pred

<tf.Tensor: shape=(3232, 3), dtype=float32, numpy=
array([[0.575742  , 0.21214055, 0.2121175 ],
       [0.57610184, 0.2119458 , 0.21195237],
       [0.57606447, 0.21195386, 0.21198173],
       ...,
       [0.21194386, 0.5761136 , 0.21194252],
       [0.21194835, 0.57610637, 0.21194525],
       [0.21196704, 0.5760251 , 0.21200784]], dtype=float32)>

In [151]:
cnn_pred1=pd.DataFrame(cnn_pred)
cnn_pred1.to_csv("cnn.csv")

In [152]:
lgbm_preds1=pd.DataFrame(lgbm_preds)
lgbm_preds1.to_csv("lgbm.csv")

In [177]:
weighted_preds = (cnn_weight* cnn_pred) + (lgbm_weight * lgbm_preds)


In [178]:
print(weighted_preds)


tf.Tensor(
[[0.5918009  0.22127877 0.18692037]
 [0.59208876 0.22112298 0.18678829]
 [0.5920589  0.22112943 0.18681177]
 ...
 [0.2812479  0.5304793  0.18827276]
 [0.28125152 0.53047353 0.18827493]
 [0.28126645 0.53040856 0.188325  ]], shape=(3232, 3), dtype=float32)


In [110]:
print(merged_data)

      Unnamed: 0  index        PTID   RID  Phase  Group  PTGENDER_-4.0  \
0              1      1  002_S_0413   413  ADNI1    0.0          False   
1              1      1  002_S_0413   413  ADNI1    0.0          False   
2              1      1  002_S_0413   413  ADNI1    0.0          False   
3              1      1  002_S_0413   413  ADNI1    0.0          False   
4              1      1  002_S_0413   413  ADNI1    0.0          False   
...          ...    ...         ...   ...    ...    ...            ...   
3227        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3228        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3229        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3230        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3231        1953   1953  136_S_1227  1227  ADNI1    1.0          False   

      PTGENDER_1.0  PTGENDER_2.0  PTWORK_-4.0  ...  PHC_MEM    PTDOB  PHC_LAN  \
0            False          Tr

NO NEED

In [64]:
merged_data.to_csv("merged.csv")

In [41]:
merged_data['Group'] = merged_data.groupby('Class_Label')['Group'].transform('first')


In [127]:
weighted_preds.shape

TensorShape([3232, 3])

HERE

In [179]:
y_pred = np.argmax(weighted_preds, axis=1)
y_pred


array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [180]:
y_true=merged_data['Group']
y_true

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3227    1.0
3228    1.0
3229    1.0
3230    1.0
3231    1.0
Name: Group, Length: 3232, dtype: float64

In [65]:
#THESE WERE CLASS COUNTS

Class_Label
AD    1124
CI    1074
CN    1034
Name: count, dtype: int64


In [181]:
conf_matrix = confusion_matrix(y_true, y_pred)

# Calculate recall for each class
num_classes = conf_matrix.shape[0]
recall_per_class = [conf_matrix[i, i] / np.sum(conf_matrix[i, :]) for i in range(num_classes)]
accuracy_per_class=[conf_matrix[i, i] / np.sum(conf_matrix[i, :]) for i in range(num_classes)]

# Calculate average recall
avg_recall = np.mean(recall_per_class)

# Print recall for each class
for i in range(num_classes):
    print(f'Recall for class {i}: {recall_per_class[i]}')
   # print(f'Accuracy for class {i}: {accuracy_per_class[i]}')

Recall for class 0: 0.9990328820116054
Recall for class 1: 0.9851024208566108
Recall for class 2: 0.998220640569395
